# Python Immersion Course

### Data Collection with Python - Part 3

### Joe Blankenship - Just some dude

Once you have a good grasp of Python's basic fucntionality, you can interact with a number of data sources. This section will focus on the basics of extracting, tranforming, and loading data formats into dataframes for analysis. Data manipulation inside of the dataframes will be saved for Part 4.

<hr>

## Web Scraping

<hr>

The first scraper we'll build will use core Python libraries to:

* Go to a HTTP website
* Gather the source code
* Print the output

In [ ]:
# Here we'll import urllib, io, and pprint modules to obtain out data

from urllib.request import Request, urlopen
from io import TextIOWrapper
from pprint import pprint

# Declare the URL
url = 'https://en.wikipedia.org/wiki/Department_of_Geography,_University_of_Kentucky'

# Open the URL
page = Request(url)
page_content = urlopen(page)
# page_content.read()

# Buffer our text stream from the website
page_data = TextIOWrapper(page_content)

# pprint out our data
for row in page_data:
    pprint(row)

However, we may want something a bit more elegant. This is where `requests` and `beautifulsoup` comes in to help us out.

In [ ]:
# Import requests and beautifulsoup
# Import pandas, we'll use that at the end
import requests
from bs4 import BeautifulSoup
import pandas as pd

# we are going to scrape crime data from the UK crime http://www.uky.edu/crimelog/
# substitute variables to fill in REST query criteria
start_month, start_day, start_year = 1, 1, 2018
end_month, end_day, end_year = 10, 4, 2018
crime_data_raw = requests.get('http://www.uky.edu/crimelog/log?field_log_category_value=All' +
                              '&field_log_report_value%5Bmin%5D%5Bmonth%5D=' + str(start_month) +
                              '&field_log_report_value%5Bmin%5D%5Bday%5D=' + str(start_day) +
                              '&field_log_report_value%5Bmin%5D%5Byear%5D=' + str(start_year) +
                              '&field_log_report_value%5Bmax%5D%5Bmonth%5D=' + str(end_month) +
                              '&field_log_report_value%5Bmax%5D%5Bday%5D=' + str(end_day) +
                              '&field_log_report_value%5Bmax%5D%5Byear%5D=' + str(end_year)
                             )


In [ ]:
# create a soup object 
crime_bs_proc = BeautifulSoup((crime_data_raw.text), "html5lib")

In [ ]:
# create a filter for our soup object to pull out the table
crime_data_table = crime_bs_proc.find('table', {'class': 'views-table cols-8'})

In [ ]:
# find the table header in the data
crime_data_header = crime_data_table.find('thead')

In [ ]:
# find all the table headers
crime_data_heads = crime_data_header.find_all('th')

In [ ]:
# create an empty list for the header
header = []

# iterate through the header element to get text
for col in crime_data_heads:
    cols = col.find_all('a')
    cols = [ele.text.strip() for ele in cols]
    header.append([ele for ele in cols if ele])

# flatten the list to a single list
header = [item for sublist in header for item in sublist]

In [ ]:
# find the table rows in the data
crime_data_body = crime_data_table.find('tbody')

In [ ]:
# find all table rows
crime_data_rows = crime_data_body.find_all('tr')

In [ ]:
# create an empty list for the rows of data
data = []

# iterate through the header element to get the rows
for row in crime_data_rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

In [ ]:
# create a dataframe with our data using our header list
uk_crime_data = pd.DataFrame(data, columns=header)
uk_crime_data.head()

There is also the `scrapy` library in Python for more complex scraping projects.

<hr>

## APIs

<hr>

APIs often have 'wrappers' in Python that you can use to interface with the underlying data.

Here we will use the data.world API to import some data

  * docs at https://github.com/datadotworld/data.world-py

Prior to this, you should load your API credentials from data.world into your active virtual env (in the terminal)

`export DW_AUTH_TOKEN=<YOUR_TOKEN>`

In [ ]:
# import our API library

import datadotworld as dw

In [ ]:
# load our data sets from the API using a known user data collection

lex_business_health = dw.load_dataset('inform8n/most-recent-lexington-ky-health-department-inspection-scores')

In [ ]:
# list the dataframes available in the data set collection

lex_business_health.dataframes

In [ ]:
# load a data set into a dataframe from the data collection

food_scores = lex_business_health.dataframes.get('most_recent_food_scores')
food_scores.head(5)

<hr>

## Flat files

<hr>

There are several ways to import flat files for analysis.

The simplest method is to use `pandas` as it supports several well known formats

However, for each of the following files, there are core and 3rd party libraries you can also use to load your data.

In [ ]:
# import pandas
import pandas as pd

In [ ]:
# read csv with pandas

census_ky_csv = pd.read_csv('data/census_2010_ky.csv')
census_ky_csv.head(2)

In [ ]:
# you can use the csv library to import/manipulate csv files

import csv

with open('data/census_2010_ky.csv') as census_ky_csv_2:
    reader = csv.DictReader(census_ky_csv_2)  # You can also use csv.reader
    for row in reader:
        print(row)

In [ ]:
# read excel in xls format with pandas

census_ky_xls = pd.read_excel('data/census_2010_ky.xls')
census_ky_xls.head(2)

In [ ]:
# read excel in xlsx format with pandas

census_ky_xlsx = pd.read_excel('data/census_2010_ky.xlsx')
census_ky_xlsx.head(2)

In [ ]:
# read json with pandas

census_ky_json = pd.read_json('data/census_2010_ky.json')
census_ky_json.head(2)

In [ ]:
# you can also use the core json library to import json data

import json

with open('data/census_2010_ky.json') as census_ky_json_2:
    reader = json.load(census_ky_json_2)
    for row in reader:
        print(row)

In [ ]:
# read xml into dataframe using core xml library

import xml.etree.ElementTree as et

root = et.parse('data/census_2010_ky.xml')  # use element tree to parse the xml data
rows = root.findall('row')  # find all row elements in xml
# iterate and select elements in row
data = [[row.find('geoid').text, row.find('label').text, row.find('totpop').text] for row in rows]
# push above data into pandas dataframe
census_ky_xml = pd.DataFrame(data, columns=['geoid', 'label', 'totpop'])
census_ky_xml.head(2)  # check your dataframe

In [ ]:
# read xml into dataframe using lxml library

from lxml import objectify
# use objectify to parse xml data
xml_data = objectify.parse(open('data/census_2010_ky.xml'))
root = xml_data.getroot()  # select root tree in xml data
# create an empty list as destination for our data
data = []
# for the row data in our root data
for elt in root.row:
    # create and empty dictionary
    el_data = {}
    # for each child element in row, extract the tag with data and append the list 'data'
    for child in elt.getchildren():
        el_data[child.tag] = child.pyval
    data.append(el_data)
# create a pandas dataframe for data list
census_ky_xml_2 = pd.DataFrame(data)
# check your dataframe
census_ky_xml_2.head(2)

In [ ]:
# import binary with pandas

census_ky_binary = pd.read_pickle('data/census_2010_ky')
census_ky_binary.head(2)

Know that `pandas` also supports many other file formats such as `hdf5`, `stata`, `SQL`, `html`, `sas`, and even data from your `clipboard`.

In [ ]:
# read pdf files... may god have mercy on your soul.

import pdfx
import pprint
# after pdfx import, create a PDFx object for our PDF
census_ky_pdf = pdfx.PDFx('data/census_2010_ky.pdf')
# extract metadata for PDF
census_ky_pdf_metadata = census_ky_pdf.get_metadata()
# extract references and place them in a dictionary, hyperlink extraction also possible
census_ky_pdf_refs = census_ky_pdf.get_references_as_dict()
# extract the body of text from PDF
census_ky_pdf_text = census_ky_pdf.reader.get_text()

This is a great starting point for extracting text, metadata, and references (with hyperlinks) from PDFs (very useful for social scientists). However, there are a few ways to extract tabular data from PDFs and none are very easy. The techniques through which the tabular text can be restructured for a dataframe will be covered in Part 4.

<hr>

## Databases

<hr>

### SQL Databases

In [ ]:
# create and write data from a sqlite3 database
# Python3 comes with sqlite and can be a power tool for initial data exploration

import sqlite3
import csv

# create the database 
connection = sqlite3.connect('data/census_2010_ky.db')
# create a cursor to interact with the database
cursor = connection.cursor()
# use the cursor to execute SQL commands on the database. here we're creating a table.
cursor.execute('CREATE TABLE kycensus2010 (id, geoid, label, totpop, medage, sindads, sinmoms);')
# with the CSV we intend to use to append out table, iterate through the CSV rows
with open('data/census_2010_ky.csv', 'r') as data:
    ky_data = csv.DictReader(data)
    to_database = [(i['id'],
                    i['geoid'],
                    i['label'],
                    i['totpop'],
                    i['medage'],
                    i['sindads'],
                    i['sinmoms'])
                   for i in ky_data
                  ]
# take the output of the above iterations and place the CSV row data into the database table
cursor.executemany('INSERT INTO kycensus2010 (id, geoid, label, totpop, medage, sindads, sinmoms) VALUES (?, ?, ?, ?, ?, ?, ?);', to_database)
# commit your changes to the database and tables
connection.commit()
# close your connection to the database
connection.close()

In [ ]:
# read data from sqlite3 database

connection = sqlite3.connect('data/census_2010_ky.db')
# use pandas to read a table from the database connection to create a dataframe
census_2010_ky_sql = pd.read_sql_query("SELECT * FROM kycensus2010", connection)
# close the database connection once you're done creating your pandas dataframe
connection.close()
# test your dataframe
census_2010_ky_sql.head(2)

In [ ]:
# use sqlalchemy to manipulate sql databases
# if you plan to move your sqlite3 database to another sql database, use this approach

import sqlalchemy as sqla
# create a sqlachemy database connection (or engine)
census_2010_ky_database = sqla.create_engine('sqlite:///data/census_2010_ky.db')
# read a table from the database usng the above engine
census_2010_ky_sql_2 = pd.read_sql('SELECT * FROM kycensus2010', census_2010_ky_database)
# test your dataframe
census_2010_ky_sql_2.head(2)

### NoSQL Databases

In [ ]:
# mongo

### Graph Databases

In [ ]:
# neo4j

<hr>

## Resources

<hr>

**Note:** A lot of the open-source materials are provided by people who develop those materials for a living. So please consider sending them a thank you and if you can, a few buck to support their efforts. Thanks! :)    

* [Pandas](https://pandas.pydata.org/pandas-docs/stable/)
* [urllib](https://docs.python.org/3/library/urllib.html)
* [io](https://docs.python.org/3/library/io.html)
* [pprint](https://docs.python.org/3/library/pprint.html)
* [requests](http://docs.python-requests.org/en/latest/)
* [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)
* [datadotworld](https://github.com/datadotworld/data.world-py)
* [csv](https://docs.python.org/3/library/csv.html)
* [json](https://docs.python.org/3/library/json.html)
* [xml](https://docs.python.org/3/library/xml.html)
* [lxml](https://lxml.de/)
* [pdfx](https://github.com/metachris/pdfx)
* [sqlite](https://docs.python.org/3/library/sqlite3.html)
* [sqlalchemy](https://docs.sqlalchemy.org/en/latest/)
* [pymongo](https://api.mongodb.com/python/current/)
* [py2neo](https://py2neo.org/v4/)